<a href="https://colab.research.google.com/github/fjadidi2001/AD_Prediction/blob/main/HippoWithResUNetandU_net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Cell 1: Setup and Imports
# Purpose: Import libraries, mount Google Drive, and set random seed.
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import ResNet50
import cv2
import zipfile
from google.colab import drive
drive.mount('/content/drive')
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)
print(f"TensorFlow version: {tf.__version__}")
print("Environment setup complete.")

Mounted at /content/drive
TensorFlow version: 2.18.0
Environment setup complete.


# Define Metrics

In [4]:
# Purpose: Define custom metrics for segmentation evaluation (Dice, IoU, precision, recall).
def dice_coef(y_true, y_pred, smooth=1e-6):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)

def iou(y_true, y_pred, smooth=1e-6):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    union = tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) - intersection
    return (intersection + smooth) / (union + smooth)

def compute_metrics(y_true, y_pred, threshold=0.5):
    y_pred_bin = (y_pred > threshold).astype(np.float32)
    dice = dice_coef(y_true, y_pred_bin).numpy()
    iou_score = iou(y_true, y_pred_bin).numpy()
    y_true_f = y_true.flatten()
    y_pred_f = y_pred_bin.flatten()
    precision = np.sum(y_true_f * y_pred_f) / (np.sum(y_pred_f) + 1e-6)
    recall = np.sum(y_true_f * y_pred_f) / (np.sum(y_true_f) + 1e-6)
    return dice, iou_score, precision, recall

print("Metrics defined: Dice, IoU, Precision, Recall")

Metrics defined: Dice, IoU, Precision, Recall


# Data Loading

In [5]:
# Purpose: Load and preprocess hippocampus MRI images and masks from zip file.
def load_image(img_path, target_size=(128, 128)):
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    img = cv2.resize(img, target_size)
    img = img / 255.0
    return img.astype(np.float32)

def load_mask(mask_path, target_size=(128, 128)):
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, target_size)
    mask = (mask > 0).astype(np.float32)
    mask = mask[:, :, np.newaxis]
    return mask

# Unzip dataset
zip_path = '/content/hippocampus segmentation dataset.zip'
extract_path = '/content/hippocampus_dataset'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Load images and masks
image_files = sorted([f for f in os.listdir(extract_path) if f.endswith('.jpg') and 'mask' not in f])
mask_files = sorted([f for f in os.listdir(extract_path) if 'mask' in f])
train_data = [(os.path.join(extract_path, img), os.path.join(extract_path, msk))
              for img, msk in zip(image_files, mask_files)]

X = np.array([load_image(img_path) for img_path, _ in train_data])
Y = np.array([load_mask(mask_path) for _, mask_path in train_data])

# Split data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=seed)
print(f"X_train shape: {X_train.shape}, X_test shape: {X_test.shape}")
print(f"Y_train shape: {Y_train.shape}, Y_test shape: {Y_test.shape}")

FileNotFoundError: [Errno 2] No such file or directory: '/content/hippocampus segmentation dataset.zip'